In [26]:
from __future__ import print_function, division, unicode_literals
import os, sys
from os.path import join
import json
from codecs import open
import nltk
import numpy as np
from nltk.corpus import stopwords
import re
import random
from itertools import chain
from time import time
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("english")

In [27]:
ROOT_DATA = join(os.environ["HOME"], "data/allen-ai-challenge")

Datasets
----

In [3]:
WIKI = join(ROOT_DATA, "parsed_wiki_data") #top5 search wiki hits from wiki
CK12 = join(ROOT_DATA, "ck12_dump") #parsing ck12
QUIZLET = join(ROOT_DATA, 'quizlet')
TOPKEK = join(ROOT_DATA, 'studystack')
KIDSDICT = join(ROOT_DATA, 'kidsdict')

TRAINING = join(ROOT_DATA, "training_set.tsv")
TRAINING_CLEANED = join(ROOT_DATA, "training_set_cleaned.tsv")
VALIDATION = join(ROOT_DATA, "validation_set.tsv")
VALIDATION_CLEANED = join(ROOT_DATA, "validation_set_cleaned.tsv")

CORPUS_PARAGRAPH = join(ROOT_DATA, "corpus_paragraph_unstable.txt")
#CORPUS_BRICKS = join(ROOT_DATA, "corpus_bricks_roman_2.txt")
#CORPUS_SENTS = join(ROOT_DATA, "corpus_sentences_roman_2.txt")

In [4]:
def tokenize(text):
    return [stemmer.stem(w) for w in nltk.word_tokenize(text.lower()) if w not in stopwords]

In [5]:
%%time
ck12_paragraphs = []
for i, fn_short in enumerate(os.listdir(CK12)):
    fn = join(CK12, fn_short)
    with open(fn, encoding='utf-8', errors='ignore') as f:
        ck12_article = json.load(f)
        for subtitle, paragraph in ck12_article['contents'].items():
            ck12_paragraphs.append(paragraph.strip())

CPU times: user 133 ms, sys: 17.3 ms, total: 150 ms
Wall time: 153 ms


In [6]:
len(ck12_paragraphs)

7148

In [7]:
%%time
wiki_paragraphs = []
for i, fn_short in enumerate(os.listdir(WIKI)):
    fn = join(WIKI, fn_short)
    with open(fn, encoding='utf-8', errors='ignore') as f:
        for line in (line.strip() for line in f):
            if not line:
                continue
            if line.startswith('=='):
                continue
            wiki_paragraphs.append(line)

CPU times: user 5.39 s, sys: 461 ms, total: 5.85 s
Wall time: 5.86 s


In [8]:
len(wiki_paragraphs)

561764

In [9]:
%%time
from StringIO import StringIO
terms = {}
for i, fn_short in enumerate(os.listdir(QUIZLET)):
    term_count = 0
    fn = join(QUIZLET, fn_short)
    with open(fn, encoding='utf-8', errors='ignore') as f:
        for line in f:
            j = json.load(StringIO(line))
            for t in j['terms']:
                terms[t['term']] = t['definition']
                term_count += 1
    print(fn_short, term_count, sep=': ')
print()
quizlet_paragraphs = [t + ' ' + d for t, d in terms.iteritems()]    

geology.txt: 211689
biology.txt: 238449
chem.txt: 186910
ck-12.txt: 9811
ck 12.txt: 9811
space.txt: 103268
lifescience.txt: 7104
earthscience.txt: 8281
climate.txt: 107490
anatomy.txt: 275689
physicalscience.txt: 4753
physiology.txt: 288464
earth science.txt: 180614
botany.txt: 247006
astronomy.txt: 190219
science.txt: 164241
life science.txt: 166535
physical science.txt: 191991
chemistry.txt: 209191
physics.txt: 196988
genetics.txt: 181100
ck12.txt: 4632
DNA.txt: 125874

CPU times: user 27.1 s, sys: 750 ms, total: 27.8 s
Wall time: 27.8 s


In [10]:
len(quizlet_paragraphs)

1192040

In [11]:
%%time
topkek_cards = []
for i, path in enumerate(os.listdir(TOPKEK)):
    fn = join(TOPKEK, path)
    with open(fn, encoding='utf-8', errors='ignore') as f:
        docs = f.readlines()
        for d in docs:
            topkek_cards.append(d.strip())
        print(path, len(docs), sep=': ')
topkek_cards = set(topkek_cards)
print()

ch_docs.txt: 426147
gen_docs.txt: 17119
es_docs.txt: 459625
bio_docs.txt: 1736698
apgeo_docs.txt: 81173
ph_docs.txt: 106345
science_docs.txt: 1094720
geo_docs.txt: 332818
anth_docs.txt: 36011
ps_docs.txt: 413622

CPU times: user 6.56 s, sys: 2.1 s, total: 8.67 s
Wall time: 8.63 s


In [12]:
len(topkek_cards)

1927951

In [13]:
%%time
kidsdict = []
for i, path in enumerate(os.listdir(KIDSDICT)):
    fn = join(KIDSDICT, path)
    with open(fn, encoding='utf-8', errors='ignore') as f:
        docs = f.readlines()
        for d in docs:
            kidsdict.append(d.strip())
        print(path, len(docs), sep=': ')
kidsdict = set(kidsdict)
print()

dict_docs.txt: 10410

CPU times: user 102 ms, sys: 4.14 ms, total: 106 ms
Wall time: 103 ms


In [14]:
len(kidsdict)

10407

Cleaning datasets
-----

In [15]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english') + '. , ! ? !? ?! ... ; : - — summary youtube www'.split())
for t in "no not above".split():
    stopwords.remove(t)

In [16]:
def text_clean(text):
    s = re.sub(r'[^\w\s\d]', '', text)
    return [stemmer.stem(w) for w in nltk.word_tokenize(s.lower()) if w not in stopwords]

In [17]:
text_clean("hello Died \\ went I Summary all of the above 45 5 www youtube")

[u'hello', u'die', u'went', u'abov', u'45', u'5']

In [46]:
%%time
c = 0
with open(CORPUS_BRICKS, encoding="utf-8", mode="w") as f:
    for d in chain(ck12_paragraphs, wiki_paragraphs, quizlet_paragraphs, topkek_cards):
        sents = list(nltk.sent_tokenize(d))
        if len(sents) == 1:
            ct = text_clean(sents[0])
            print(*ct, sep=" ", file=f)
            c += 1
        else:
            for s1, s2 in zip(sents[:-1], sents[1:]):
                ct = text_clean(s1 + ' ' + s2)
                print(*ct, sep=" ", file=f)
                c += 1
print('Wrote', c, 'bricks')

KeyboardInterrupt: 

In [ ]:
%%time
c = 0
with open(CORPUS_SENTS, encoding="utf-8", mode="w") as f:
    for d in chain(ck12_paragraphs, wiki_paragraphs, quizlet_paragraphs, topkek_cards):
        for s in nltk.sent_tokenize(d):
            ct = text_clean(s)
            print(*ct, sep=" ", file=f)
            c += 1
print('Wrote', c, 'sents')

In [ ]:
%%time
c = 0
n = 50
overlap = 10
with open(CORPUS_PARAGRAPH, encoding="utf-8", mode="w") as f:
    for d in chain(ck12_paragraphs, wiki_paragraphs,
                   quizlet_paragraphs, topkek_cards, kidsdict):
        ct = text_clean(d)
        if len(ct) >= 100:
            for q in [ct[i:i+n] for i in range(0, len(ct) - n, n-overlap)]:
                print(*q, sep=" ", file=f)
                c += 1
        else:
            print(*ct, sep=" ", file=f)
            c += 1
print('Wrote', c, 'paragraphs')

Train/validation
-----

In [ ]:
def text_clean_join(t):
    return " ".join(text_clean(t))

In [ ]:
with open(TRAINING_CLEANED, encoding="utf-8", mode="w") as fo:
    with open(TRAINING, encoding="utf-8") as f:
        next(f)
        for i, l in enumerate(f):
            [qid, q, r, aa, ab, ac, ad] = l.strip().split("\t")
            print(qid, text_clean_join(q), r,
                  text_clean_join(aa),
                  text_clean_join(ab),
                  text_clean_join(ac),
                  text_clean_join(ad),
                  sep="\t", file=fo)

In [ ]:
with open(VALIDATION_CLEANED, encoding="utf-8", mode="w") as fo:
    with open(VALIDATION, encoding="utf-8") as f:
        next(f)
        for i, l in enumerate(f):
            [qid, q, aa, ab, ac, ad] = l.strip().split("\t")
            print(qid, text_clean_join(q),
                  text_clean_join(aa),
                  text_clean_join(ab),
                  text_clean_join(ac),
                  text_clean_join(ad),
                  sep="\t", file=fo)

In [ ]:
'+'.join('104080  germin compos mollusk kingdom   scallop protista        dna molecul     '.split())